In [1]:
import sys
sys.executable


'/home/pia/Documents/distillistic/venv/bin/python'

In [2]:
import os
import time
from enum import Enum

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from distillistic import resnet18


In [3]:
# Data loading code
data_dir = "/home/pia/Documents/ImageNet"
traindir = os.path.join(data_dir, 'train')
valdir = os.path.join(data_dir, 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
val_dataset = datasets.ImageFolder(
    valdir,
    transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))

workers = 2
bs = 2

train_loader = DataLoader(
    train_dataset, batch_size=bs, shuffle=True,
    num_workers=workers, pin_memory=True)

val_loader = DataLoader(
    val_dataset, batch_size=bs, shuffle=False,
    num_workers=workers, pin_memory=True)


In [4]:
len(train_dataset), len(val_dataset)


(1281167, 50000)

In [5]:
model = resnet18(1000, pretrained=True)
criterion = torch.nn.CrossEntropyLoss()


In [6]:
class Summary(Enum):
    NONE = 0
    AVERAGE = 1
    SUM = 2
    COUNT = 3


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, name, fmt=':f', summary_type=Summary.AVERAGE):
        self.name = name
        self.fmt = fmt
        self.summary_type = summary_type
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

    def summary(self):
        fmtstr = ''
        if self.summary_type is Summary.NONE:
            fmtstr = ''
        elif self.summary_type is Summary.AVERAGE:
            fmtstr = '{name} {avg:.3f}'
        elif self.summary_type is Summary.SUM:
            fmtstr = '{name} {sum:.3f}'
        elif self.summary_type is Summary.COUNT:
            fmtstr = '{name} {count:.3f}'
        else:
            raise ValueError('invalid summary type %r' % self.summary_type)

        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def display_summary(self):
        entries = [" *"]
        entries += [meter.summary() for meter in self.meters]
        print(' '.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


In [9]:
def validate(val_loader, model, criterion, gpu=None):
    batch_time = AverageMeter('Time', ':6.3f', Summary.NONE)
    losses = AverageMeter('Loss', ':.4e', Summary.NONE)
    top1 = AverageMeter('Acc@1', ':6.2f', Summary.AVERAGE)
    top5 = AverageMeter('Acc@5', ':6.2f', Summary.AVERAGE)
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if gpu is not None:
                images = images.cuda(gpu, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(gpu, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 1000 == 0:
                progress.display(i)

        progress.display_summary()

    return top1.avg


In [10]:
validate(val_loader, model, criterion)


Test: [    0/25000]	Time  0.164 ( 0.164)	Loss 1.3228e+00 (1.3228e+00)	Acc@1  50.00 ( 50.00)	Acc@5 100.00 (100.00)
Test: [ 1000/25000]	Time  0.060 ( 0.058)	Loss 7.1779e-01 (7.7398e-01)	Acc@1  50.00 ( 80.52)	Acc@5 100.00 ( 93.91)
Test: [ 2000/25000]	Time  0.059 ( 0.059)	Loss 3.4904e-02 (1.0058e+00)	Acc@1 100.00 ( 74.26)	Acc@5 100.00 ( 91.90)
Test: [ 3000/25000]	Time  0.061 ( 0.059)	Loss 1.6939e+00 (9.2558e-01)	Acc@1  50.00 ( 76.59)	Acc@5 100.00 ( 92.52)
Test: [ 4000/25000]	Time  0.060 ( 0.059)	Loss 1.6000e-02 (8.7207e-01)	Acc@1 100.00 ( 77.81)	Acc@5 100.00 ( 92.89)
Test: [ 5000/25000]	Time  0.045 ( 0.059)	Loss 2.9407e-01 (9.1488e-01)	Acc@1 100.00 ( 76.41)	Acc@5 100.00 ( 92.90)
Test: [ 6000/25000]	Time  0.061 ( 0.059)	Loss 2.3771e+00 (9.1782e-01)	Acc@1   0.00 ( 76.01)	Acc@5 100.00 ( 93.04)
Test: [ 7000/25000]	Time  0.059 ( 0.059)	Loss 1.0491e-01 (9.1334e-01)	Acc@1 100.00 ( 75.73)	Acc@5 100.00 ( 93.34)
Test: [ 8000/25000]	Time  0.059 ( 0.059)	Loss 8.5245e-02 (9.2315e-01)	Acc@1 100.00 ( 75.

KeyboardInterrupt: 